In [13]:
import cv2
import numpy as np

In [14]:
def resize_and_crop_image(image):
    ratio = 1
    max_edge = max(image.shape[0], image.shape[1])
    while max_edge > 1000:
        ratio *= 2
        max_edge //= 2

    rows = image.shape[0] // ratio
    cols = image.shape[1] // ratio

    # print(f"resize ratio = {ratio}.")
    image_resized = cv2.resize(image, (cols, rows))
    # print(image_resized.shape)
    if CROP:
        size = min(cols, rows)
        return image_resized[:size, :size]
    else:
        return image_resized

In [15]:
def RGBRange(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 48, 80], dtype="uint8")
    upper_skin = np.array([20, 255, 255], dtype="uint8")
    skinMask = cv2.inRange(hsv, lower_skin, upper_skin)
    skin = cv2.bitwise_and(image, image, mask=skinMask)
    return skin

In [16]:
EXP = True
CROP = False

In [17]:
mode = "rgb"
process_method = None
if mode == "rgb":
    process_method = RGBRange
    
    
assert process_method is not None

def process_func(image):
    return process_method(resize_and_crop_image(image))


In [18]:
import os

# Define paths
VIDEO_NAME = "fingers"
FVideo = f'videos/{VIDEO_NAME}.mp4'
WORKDIR = f"videos/{VIDEO_NAME}/"
os.makedirs(WORKDIR, exist_ok=True)
FFirst = WORKDIR + "first.png"  # Filename for the first frame image
FOutput = WORKDIR + f"{mode}.mp4"  # Filename for the output video

In [19]:
# Create video capture object
vidCap = cv2.VideoCapture(FVideo)

# Get video properties for the output video

fps = vidCap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    

# Process video frames
success, image = vidCap.read()
assert success

cv2.imwrite(FFirst, image)

skin = process_func(image)
# Show the result
cv2.imshow("Skin detection", skin)
cv2.waitKey(0)
cv2.destroyAllWindows()
        
if not EXP:
    # frame_width = int(vidCap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # frame_height = int(vidCap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width = skin.shape[1]
    frame_height = skin.shape[0]
    out = cv2.VideoWriter(FOutput, fourcc, fps, (frame_width, frame_height))
    
    while success:
        # Apply skin detection
        skin = process_func(image)
        # Write the processed frame
        out.write(skin)
        success, image = vidCap.read()

    out.release()
    

# Release resources
vidCap.release()
cv2.destroyAllWindows()
